# NDVI Calculation from Satellite Imagery - Greece
## SSP25 Project: Post-fire biomass and wildlife recovery

This notebook demonstrates how to get satellite imagery from Greece and calculate NDVI (Normalized Difference Vegetation Index) using Google Earth Engine.

**NDVI Formula:** `(NIR - Red) / (NIR + Red)`

**NDVI Values:**
- **-1 to 0**: Water, bare soil, rock
- **0 to 0.3**: Sparse vegetation
- **0.3 to 0.6**: Moderate vegetation
- **0.6 to 1**: Dense, healthy vegetation

## Step 1: Install Required Packages
Run this cell first if packages are not installed:

In [1]:
# Uncomment and run if packages are not installe
!pip install geemap earthengine-api
!pip install rasterio matplotlib numpy

## Step 2: Import Libraries and Initialize Earth Engine

In [9]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
ee.Authenticate()
ee.Initialize(project='ee-sandorburian')

In [10]:
# Initialize Earth Engine (you need to authenticate first)
# Run ee.Authenticate() once if this is your first time using GEE
try:
    ee.Initialize()
    print("✓ Earth Engine initialized successfully")
except Exception as e:
    print(f"Error: {e}")
    print("Run: ee.Authenticate() first")

Error: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.
Run: ee.Authenticate() first


## Step 3: Define Greece Study Area

In [45]:
# Method 1: Use country boundaries from Earth Engine
greece = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017") \
           .filter(ee.Filter.eq('country_na', 'Greece'))
greece_boundary = greece.geometry()

# Method 2: Or define a specific region (e.g., area affected by wildfires)
# Example: Area around Athens (you can modify coordinates)
athens_region = ee.Geometry.Rectangle([23.5, 37.8, 24.1, 38.2])
test_region   = ee.Geometry.Rectangle([40.7, 26.4, 24.1, 38.2])

burn_test_region = ee.Geometry.Rectangle([
    25.419968028200316, 40.79514759494151,  # xmin, ymin (SW corner)
    26.305033189080742, 41.2277677156888    # xmax, ymax (NE corner)
])

# Choose your area of interest
area_of_interest = burn_test_region #athens_region #greece_boundary  # Use greece_boundary or athens_region

print("✓ Study area defined for Greece")

✓ Study area defined for Greece


## Step 4: Get Sentinel-2 Satellite Imagery

In [62]:
# Define time period (modify dates as needed)
start_date = '2023-07-13' #'2023-06-01'  # Summer period for better vegetation analysis
end_date = '2023-08-13' #28' #'2023-08-31'
cloud_threshold = 10  # Maximum cloud cover percentage

# Get Sentinel-2 Surface Reflectance collection
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
              .filterDate(start_date, end_date) \
              .filterBounds(area_of_interest) \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_threshold))

# Check how many images we found
image_count = sentinel2.size().getInfo()
print(f"Found {image_count} Sentinel-2 images for the specified period")

if image_count == 0:
    print("⚠️  No images found. Try expanding date range or increasing cloud threshold.")

Found 15 Sentinel-2 images for the specified period


# 4.2 Get LANDSAT images



In [100]:
# Define time period (modify dates as needed)
start_date = '2011-08-24'  # Summer period for better vegetation analysis
end_date = '2011-10-24' #28
cloud_threshold = 90  # Maximum cloud cover percentage

# Get Sentinel-2 Surface Reflectance collection
sentinel2 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
              .filterDate(start_date, end_date) \
              .filterBounds(area_of_interest) # \
              #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_threshold))

# Check how many images we found
image_count = sentinel2.size().getInfo()
print(f"Found {image_count} USGS Landsat 7 Level 2, Collection 2, Tier 1 images for the specified period")

if image_count == 0:
    print("⚠️  No images found. Try expanding date range or increasing cloud threshold.")

Found 16 USGS Landsat 7 Level 2, Collection 2, Tier 1 images for the specified period


## Step 5: Calculate NDVI

In [63]:
def calculate_ndvi(image):
    """
    Calculate NDVI for a Sentinel-2 image
    B4 = Red band, B8 = NIR band
    """
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Apply NDVI calculation to all images
ndvi_collection = sentinel2.map(calculate_ndvi)

# Create median composite to reduce cloud effects and seasonal variation
median_ndvi = ndvi_collection.select('NDVI').median().clip(area_of_interest)

print("✓ NDVI calculated successfully")

✓ NDVI calculated successfully


## Step 6: Visualize NDVI on Interactive Map

In [64]:
# Create interactive map centered on Greece
Map = geemap.Map(center=[39.0, 22.0], zoom=6)

# Define NDVI visualization parameters
ndvi_params = {
    'min': -0.2,
    'max': 0.8,
    'palette': [
        '#d73027',  # Red (low NDVI - bare soil/water)
        '#f46d43',  # Orange
        '#fdae61',  # Light orange
        '#fee08b',  # Yellow
        '#e6f598',  # Light green
        '#abdda4',  # Green
        '#66c2a5',  # Dark green
        '#3288bd'   # Blue-green (high NDVI - dense vegetation)
    ]
}

# Add NDVI layer to map
Map.addLayer(median_ndvi, ndvi_params, 'NDVI Greece')

# Add Greece boundary (optional)
Map.addLayer(greece_boundary, {'color': 'red', 'width': 2}, 'Greece Boundary', False)

# Center map on area of interest
Map.centerObject(area_of_interest, 6)

# Display map
Map

Map(center=[41.012067138890096, 25.862500608640715], controls=(WidgetControl(options=['position', 'transparent…

## Step 7: Get NDVI Statistics

In [49]:
# Calculate NDVI statistics for the study area
ndvi_stats = median_ndvi.reduceRegion(
    reducer=ee.Reducer.mean().combine(
        ee.Reducer.min(), sharedInputs=True).combine(
        ee.Reducer.max(), sharedInputs=True).combine(
        ee.Reducer.stdDev(), sharedInputs=True),
    geometry=area_of_interest,
    scale=100,  # 100m resolution
    maxPixels=1e9
).getInfo()

print("📊 NDVI Statistics for Greece:")
print(f"   Mean NDVI: {ndvi_stats['NDVI_mean']:.3f}")
print(f"   Min NDVI:  {ndvi_stats['NDVI_min']:.3f}")
print(f"   Max NDVI:  {ndvi_stats['NDVI_max']:.3f}")
print(f"   Std Dev:   {ndvi_stats['NDVI_stdDev']:.3f}")

# Interpret results
mean_ndvi = ndvi_stats['NDVI_mean']
if mean_ndvi > 0.6:
    interpretation = "Dense, healthy vegetation"
elif mean_ndvi > 0.3:
    interpretation = "Moderate vegetation cover"
elif mean_ndvi > 0:
    interpretation = "Sparse vegetation"
else:
    interpretation = "Predominantly non-vegetated areas"

print(f"🌱 Interpretation: {interpretation}")

📊 NDVI Statistics for Greece:
   Mean NDVI: 0.516
   Min NDVI:  -0.383
   Max NDVI:  0.923
   Std Dev:   0.269
🌱 Interpretation: Moderate vegetation cover


## Step 8: Export NDVI Data (Optional)

In [50]:
# Export NDVI to Google Drive
export_task = ee.batch.Export.image.toDrive(
    image=median_ndvi,
    description='Greece_NDVI_2023',
    folder='EarthEngine',
    fileNamePrefix='greece_ndvi_2023',
    region=area_of_interest,
    scale=100,  # 100m resolution
    crs='EPSG:4326',
    maxPixels=1e9
)

# Start the export task
export_task.start()

print("✓ Export task started. Check your Google Drive 'EarthEngine' folder.")
print("✓ File will be saved as 'greece_ndvi_2023.tif'")

✓ Export task started. Check your Google Drive 'EarthEngine' folder.
✓ File will be saved as 'greece_ndvi_2023.tif'


## Bonus: Post-Fire Analysis Function
For the SSP25 project on post-fire recovery:

In [84]:
def analyze_fire_recovery(fire_area, before_fire_date, after_fire_date):
    """
    Analyze vegetation recovery after wildfire

    Parameters:
    - fire_area: ee.Geometry of the burned area
    - before_fire_date: Date before fire (YYYY-MM-DD)
    - after_fire_date: Date after fire (YYYY-MM-DD)
    """
    # Define a date range around the before and after dates
    before_start = ee.Date(before_fire_date).advance(-1, 'month')
    before_end = ee.Date(before_fire_date).advance(1, 'month')
    after_start = ee.Date(after_fire_date).advance(-1, 'month')
    after_end = ee.Date(after_fire_date).advance(1, 'month')


    # Get NDVI before fire
    before_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                          .filterDate(before_start, before_end) \
                          .filterBounds(fire_area) \
                          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

    ndvi_before = before_collection.map(calculate_ndvi).select('NDVI').median()

    # Get NDVI after fire
    after_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                         .filterDate(after_start, after_end) \
                         .filterBounds(fire_area) \
                         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

    ndvi_after = after_collection.map(calculate_ndvi).select('NDVI').median()

    # Calculate recovery (difference)
    ndvi_recovery = ndvi_after.subtract(ndvi_before).rename('Recovery')

    return ndvi_before, ndvi_after, ndvi_recovery

# Example usage (uncomment and modify coordinates):
fire_area = ee.Geometry.Rectangle([
    25.419968028200316, 40.79514759494151,  # xmin, ymin (SW corner)
    26.305033189080742, 41.2277677156888    # xmax, ymax (NE corner)
])

#([23.0, 38.0, 24.0, 39.0])  # Example fire area
##                                                           Before fire  After fire (2 years later)
before, after, recovery = analyze_fire_recovery(fire_area, '2023-07-14', '2023-08-28')

print("🔥 Fire recovery analysis function ready!")
print("   Modify the example to analyze specific burned areas.")
recovery

🔥 Fire recovery analysis function ready!
   Modify the example to analyze specific burned areas.


## Recovery based on LANDSAT images
https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C02_T1_L2#colab-python

In [106]:
def calculate_ndvi_landsat7(image):
    # Landsat 7 SR_B4 = NIR, SR_B3 = Red
    ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
    return image.addBands(ndvi)

In [116]:
def analyze_fire_recovery_LANDDSAT(fire_area, before_fire_date, after_fire_date):
    """
    Analyze vegetation recovery after wildfire

    Parameters:
    - fire_area: ee.Geometry of the burned area
    - before_fire_date: Date before fire (YYYY-MM-DD)
    - after_fire_date: Date after fire (YYYY-MM-DD)
    """
    # Define a date range around the before and after dates
    before_start = ee.Date(before_fire_date).advance(-1, 'month')
    before_end = ee.Date(before_fire_date).advance(1, 'month')
    after_start = ee.Date(after_fire_date).advance(-1, 'month')
    after_end = ee.Date(after_fire_date).advance(1, 'month')


    # Get NDVI before fire
    before_collection = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
                          .filterDate(before_start, before_end) \
                          .filterBounds(fire_area)# \
                          #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

    ndvi_before = before_collection.map(calculate_ndvi_landsat7).select('NDVI').median()

    # Get NDVI after fire
    after_collection = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
                         .filterDate(after_start, after_end) \
                         .filterBounds(fire_area)# \
                         #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

    ndvi_after = after_collection.map(calculate_ndvi_landsat7).select('NDVI').median()

    # Calculate recovery (difference)
    ndvi_recovery = ndvi_after.subtract(ndvi_before).rename('Recovery')

    #print("before_collection",before_collection)
    return ndvi_before, ndvi_after, ndvi_recovery

# Example usage (uncomment and modify coordinates):
fire_area = ee.Geometry.Rectangle([
    25.419968028200316, 40.79514759494151,  # xmin, ymin (SW corner)
    26.305033189080742, 41.2277677156888    # xmax, ymax (NE corner)
])

before, after, recovery_landsat = analyze_fire_recovery_LANDDSAT(fire_area, '2011-08-23', '2011-08-28')

print("🔥 Fire recovery analysis function ready!")
print("   Modify the example to analyze specific burned areas.")
recovery_landsat

🔥 Fire recovery analysis function ready!
   Modify the example to analyze specific burned areas.


In [117]:
# Visualize the recovery image on the map

# Create an interactive map centered on the fire area
Map_recovery = geemap.Map(center=[41.142456, 25.975859], zoom=8) # Center on example fire area

# Define visualization parameters for recovery (difference)
# Positive values indicate recovery (NDVI increased), negative values indicate decrease
recovery_params = {
    'min': -0.5,  # Significant decrease in NDVI
    'max': 0.5,   # Significant increase in NDVI
    'palette': [
        '#d7191c',  # Red (decrease)
        '#fdae61',  # Orange (slight decrease)
        '#ffffbf',  # Yellow (no change)
        '#a6d96a',  # Light green (slight increase)
        '#1a9641'   # Dark green (increase)
    ]
}

# Add recovery layer to map
#Map_recovery.addLayer(recovery, recovery_params, 'NDVI Recovery')
Map_recovery.addLayer(recovery_landsat, recovery_params, 'NDVI Recovery')

# Add the fire area boundary (optional)
Map_recovery.addLayer(fire_area, {'color': 'blue', 'width': 2}, 'Fire Area Boundary', True)

# Add a legend to the map
Map_recovery.add_legend(title="NDVI Recovery", legend_dict={
    'Significant Decrease': '#d7191c',
    'Slight Decrease': '#fdae61',
    'No Change': '#ffffbf',
    'Slight Increase': '#a6d96a',
    'Significant Increase': '#1a9641'
})

# Display map
Map_recovery

Map(center=[41.142456, 25.975859], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

## Summary

This notebook demonstrated how to:
1. ✅ Set up Google Earth Engine for satellite data access
2. ✅ Define study areas in Greece
3. ✅ Filter and process Sentinel-2 imagery
4. ✅ Calculate NDVI using the formula (NIR-Red)/(NIR+Red)
5. ✅ Visualize results on interactive maps
6. ✅ Extract statistics and interpret vegetation health
7. ✅ Export data for further analysis

**Next Steps for Post-Fire Research:**
- Compare NDVI before/after wildfire events
- Monitor vegetation recovery over time
- Analyze different vegetation types and recovery rates
- Combine with wildlife habitat data

**Key Learning Points:**
- NDVI ranges from -1 to 1
- Higher values = healthier vegetation
- Sentinel-2 provides 10m resolution data
- Cloud filtering is essential for accurate results
- Median composites reduce noise and gaps

In [69]:
# Visualize the recovery image on the map

# Create an interactive map centered on the fire area
Map_recovery = geemap.Map(center=[41.142456, 25.975859], zoom=8) # Center on example fire area

# Define visualization parameters for recovery (difference)
# Positive values indicate recovery (NDVI increased), negative values indicate decrease
recovery_params = {
    'min': -0.5,  # Significant decrease in NDVI
    'max': 0.5,   # Significant increase in NDVI
    'palette': [
        '#d7191c',  # Red (decrease)
        '#fdae61',  # Orange (slight decrease)
        '#ffffbf',  # Yellow (no change)
        '#a6d96a',  # Light green (slight increase)
        '#1a9641'   # Dark green (increase)
    ]
}

# Add recovery layer to map
Map_recovery.addLayer(recovery, recovery_params, 'NDVI Recovery')

# Add the fire area boundary (optional)
Map_recovery.addLayer(fire_area, {'color': 'blue', 'width': 2}, 'Fire Area Boundary', True)

# Add a legend to the map
Map_recovery.add_legend(title="NDVI Recovery", legend_dict={
    'Significant Decrease': '#d7191c',
    'Slight Decrease': '#fdae61',
    'No Change': '#ffffbf',
    'Slight Increase': '#a6d96a',
    'Significant Increase': '#1a9641'
})

# Display map
Map_recovery

Map(center=[41.142456, 25.975859], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…